In [ ]:
from pinecone import Pinecone, ServerlessSpec
import pickle
import os

pinecone = Pinecone(
   api_key = os.getenv['PINECONE_API_KEY']
)

pinecone = Pinecone(
   api_key = os.getenv['PINECONE_API_KEY']
)

my_index_name = "pinterest-multimodal-search"
my_index = pinecone.Index(name = my_index_name)

In [ ]:
# initialise the imagebind modal
from models import data
import torch
from models import imagebind_model
from models.imagebind_model import ModalityType

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = imagebind_model.imagebind_huge(pretrained=True)

In [ ]:
# using Textto Image
query = "blue jeans with 4 pockets"
inputs = {
            ModalityType.TEXT: data.load_and_transform_text(query, device)
        }

with torch.no_grad():
    embeddings = model(inputs)

query_embedding = embeddings[ModalityType.TEXT].numpy()
result = my_index.query(vector=query_embedding.tolist()[0], top_k=topk, include_metadata=True)

In [ ]:
for i in range(topk):
    img_url = result['matches'][i]['metadata']['image_url']
    response = requests.get(img_url)
    if response.status_code == 200:
        # Read the image from the response content
        image = Image.open(BytesIO(response.content))
        # Display the image
        plt.imshow(image)
        plt.axis('off')  # Turn off axis
        plt.show()
    else:
        print("Failed to fetch image from URL")

In [ ]:
#Using Image to Image
# Using Images
img_query = ['images/f68b296900cc4166c7e2dde9445fc8fd.jpg']
inputs = {
            ModalityType.VISION: data.load_and_transform_vision_data(img_query, device)
        }

with torch.no_grad():
    embeddings = model(inputs)

query_embedding = embeddings[ModalityType.VISION].numpy()
result = my_index.query(vector=query_embedding.tolist()[0], top_k=3, include_metadata=True)

In [ ]:
for i in range(topk):
    img_url = result['matches'][i]['metadata']['image_url']
    response = requests.get(img_url)
    if response.status_code == 200:
        # Read the image from the response content
        image = Image.open(BytesIO(response.content))
        # Display the image
        plt.imshow(image)
        plt.axis('off')  # Turn off axis
        plt.show()
    else:
        print("Failed to fetch image from URL")

In [ ]:
# Audio to Image
import torchaudio
platform = torchaudio.get_audio_backend()

if platform == 'sox' or platform == 'sox_io':
    # Set the backend to 'sox_io' on Linux/macOS
    torchaudio.set_audio_backend('sox_io')
elif platform == 'soundfile':
    # Set the backend to 'soundfile' on Windows
    torchaudio.set_audio_backend('soundfile')
else:
    # Handle unsupported platforms or cases where no backend is available
    print("No suitable audio backend available for the current platform.")

In [ ]:
# Audio to Image models
from models import data
import torch
from models.imagebind_model import ModalityType
audio_query = ['audio/blue_jeans_1.wav']

device = "cuda:0" if torch.cuda.is_available() else "cpu"

inputs = {
            ModalityType.AUDIO: data.load_and_transform_audio_data(audio_query, device)
        }

with torch.no_grad():
    embeddings = model(inputs)

query_embedding = embeddings[ModalityType.AUDIO].numpy()
result = my_index.query(vector=query_embedding.tolist()[0], top_k=5, include_metadata=True)

In [ ]:
for i in range(topk):
    img_url = result['matches'][i]['metadata']['image_url']
    response = requests.get(img_url)
    if response.status_code == 200:
        # Read the image from the response content
        image = Image.open(BytesIO(response.content))
        # Display the image
        plt.imshow(image)
        plt.axis('off')  # Turn off axis
        plt.show()
    else:
        print("Failed to fetch image from URL")